In [38]:
import pandas as pd
import psycopg2
import bokeh.plotting
import bokeh.models as bmodels
import bokeh.plotting as plt
import bokeh.palettes as bpalettes
import bokeh.transform as btransform
import bokeh.io

import server.model.connection as smc
import server.model.version as smv
import server.model.event as sme

bokeh.io.output_notebook()

Loading BokehJS ...

In [39]:
conn = smc.pool.getconn()

In [51]:
match = '020-q'
alliance = 'red'
sql = '''
    SELECT team, task, SUM(successes) AS hatches_placed FROM vw_measures 
    WHERE task IN ('centerHatch', 'farHatch', 'closeHatch', 'rocketHatch',
    'rocketHatch1', 'rocketHatch2', 'rocketHatch3', 'csHatch')
    AND team IN (SELECT team FROM schedules
                 INNER JOIN status ON schedules.event_id=status.event_id 
                 WHERE schedules.event_id=status.event_id AND schedules.match=%s
                 AND alliance=%s)
    GROUP BY team, task;
'''
df_temp = pd.read_sql(sql, conn, params=[match, alliance])
#df_redHatches = df_temp.set_index('team')
df_temp

,team,task,hatches_placed
0,1258,centerHatch,1
1,1258,closeHatch,1
2,1258,csHatch,6
3,1258,rocketHatch1,7
4,1258,rocketHatch2,4
5,1258,rocketHatch3,2
6,3786,csHatch,6
7,3786,rocketHatch,1
8,3786,rocketHatch1,1
9,3786,rocketHatch3,1


In [52]:
df_temp = df_temp.set_index(['task', 'team'])
df_temp

,,hatches_placed
task,team,
centerHatch,1258,1
closeHatch,1258,1
csHatch,1258,6
rocketHatch1,1258,7
rocketHatch2,1258,4
rocketHatch3,1258,2
csHatch,3786,6
rocketHatch,3786,1
rocketHatch1,3786,1


In [53]:
df_temp = df_temp.unstack()
df_temp

hatches_placed          
team                   1258 3786 4131
task                                 
centerHatch             1.0  NaN  NaN
closeHatch              1.0  NaN  NaN
csHatch                 6.0  6.0  8.0
rocketHatch             NaN  1.0  1.0
rocketHatch1            7.0  1.0  3.0
rocketHatch2            4.0  NaN  3.0
rocketHatch3            2.0  1.0  1.0

In [54]:
df_temp.columns = df_temp.columns.droplevel()


In [55]:
df_temp

team,1258,3786,4131
task,,,
centerHatch,1.0,NaN,NaN
closeHatch,1.0,NaN,NaN
csHatch,6.0,6.0,8.0
rocketHatch,NaN,1.0,1.0
rocketHatch1,7.0,1.0,3.0
rocketHatch2,4.0,NaN,3.0
rocketHatch3,2.0,1.0,1.0


In [57]:
rHatches_cds = bmodels.ColumnDataSource(df_temp)
rHatches_cds.column_names

['task', '1258', '3786', '4131']

In [58]:
rteams = rHatches_cds.column_names[1:]
rteams

['1258', '3786', '4131']

In [60]:
p = plt.figure(title='Hatches Placed', x_range=['task'], 
              plot_width=700, plot_height=350)
p.vbar_stack(rteams, x='index', width=0.4, 
            source=rHatches_cds, color=bpalettes.Reds3, legend=[" " + x for x in rteams])

bokeh.io.show(p)

ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name. This could either be due to a misspelling or typo, or due to an expected column being missing. : key "x" value "index" [renderer: GlyphRenderer(id='1868', ...)]
ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name. This could either be due to a misspelling or typo, or due to an expected column being missing. : key "x" value "index" [renderer: GlyphRenderer(id='1881', ...)]
ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name. This could either be due to a misspelling or typo, or due to an expected column being missing. : key "x" value "index" [renderer: GlyphRenderer(id='1895', ...)]
